In [ ]:
city  = {'Quito': 1, 
        'Santo Domingo': 2, 
        'Cayambe': 3,
        'Latacunga': 4,
        'Riobamba': 5,
        'Ibarra': 6, 
        'Guaranda': 7, 
        'Puyo': 8, 
        'Ambato': 9, 
        'Guayaquil': 10, 
        'Salinas': 11,
        'Daule': 12, 
        'Babahoyo': 13, 
        'Quevedo': 14, 
        'Playas': 15, 
        'Libertad': 16, 
        'Cuenca': 17,
        'Loja': 18, 
        'Machala': 19, 
        'Esmeraldas': 20, 
        'Manta': 21, 
        'El Carmen': 22}
state = {'Pichincha': 31, 
         'Santo Domingo de los Tsachilas': 32, 
         'Cotopaxi': 33,
         'Chimborazo': 34, 
         'Imbabura': 35, 
         'Bolivar': 36, 
         'Pastaza': 37, 
         'Tungurahua': 38,
         'Guayas': 39, 
         'Santa Elena': 40, 
         'Los Rios': 41, 
         'Azuay': 42, 
         'Loja': 43, 
         'El Oro': 44,
         'Esmeraldas': 45, 
         'Manabi': 46,
         'Ecuador': 50}

store_type = {'A': 1, 
              'B': 2, 
              'C': 3,
              'D': 4, 
              'E': 5}

In [1]:
import pandas as pd
import numpy as np
import pickle
# import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')
import gc
import lightgbm as lgb

# Предобработка

#### Считываем данные

In [3]:
%%time

types = {'id': np.uint32,
         'item_nbr': np.uint32,
         'store_nbr': np.uint8,
         'onpromotion': 'bool'
        }
train = pd.read_csv('data/train.csv.bz2', compression='bz2', index_col='id', parse_dates = ['date'], dtype = types, 
                    infer_datetime_format = True, skiprows=range(1, 50275926),    # since 2015-07-01 плюс два ряда к id 
                    converters={'unit_sales': lambda x: np.log1p(float(x)) if float(x) > 0 else 0})

# Прологарифмировали, уменьшаем тип данных
train['unit_sales'] = train['unit_sales'].astype(np.float16)
train['onpromotion'] = train['onpromotion'].map({False : 0, True : 1})
train['onpromotion'] = train['onpromotion'].astype(np.uint8)

CPU times: user 8min 7s, sys: 15.1 s, total: 8min 22s
Wall time: 8min 22s


In [4]:
%%time
types = {'id': np.uint32,
         'item_nbr': np.uint32,
         'store_nbr': np.uint8,
         'onpromotion': 'bool'
        }
test = pd.read_csv('data/test.csv.gz', compression='gzip', index_col='id', parse_dates = ['date'], dtype = types, 
                    infer_datetime_format = True)

test['onpromotion'] = test['onpromotion'].map({False : 0, True : 1})
test['onpromotion'] = test['onpromotion'].astype(np.uint8)

CPU times: user 4.84 s, sys: 32 ms, total: 4.88 s
Wall time: 4.88 s


#### Удаляем ненужные товары-магазины

In [5]:
%%time
# Удаляем пары магазин-товар, которые не были замечены в трейне с 2017.06.01
last_unique = train[train['date'] >= '2017.06.01'].filter(items=['store_nbr', 'item_nbr'])

last_unique = last_unique.drop_duplicates()

test = test.merge(last_unique, indicator=True, how='left')
test = test[test._merge=='both']
test.drop('_merge', axis=1, inplace=True)
print(test.shape)

del last_unique

(2486416, 4)
CPU times: user 3.35 s, sys: 1.86 s, total: 5.2 s
Wall time: 5.47 s


In [6]:
%%time
# Удаляем пары магазин-товар, которые не были замечены в трейне с 2017.06.01
# Потом предикт надо слить с сабмитом по индексу и заполнить нулями NA
last_unique = test.filter(items=['store_nbr', 'item_nbr'])

last_unique = last_unique.drop_duplicates()

train = train.merge(last_unique, indicator=True, how='left')
train = train[train._merge=='both']
train.drop('_merge', axis=1, inplace=True)
print(train.shape)

del last_unique

(71448006, 5)
CPU times: user 58.4 s, sys: 42.1 s, total: 1min 40s
Wall time: 1min 40s


In [6]:
train.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2015-07-01,1,103520,1.946289,1
1,2015-07-01,1,103665,1.609375,0
2,2015-07-01,1,105574,1.609375,0
3,2015-07-01,1,105575,2.833984,0
4,2015-07-01,1,105737,1.386719,0


In [10]:
%%time
# Удаляем в трейне товары, которых нет в тесте
# Предварительно в тесте удаляем пары магазин-товар, которые не были замечены в трейне с 2017.06.01
items_to_del = list(set(train['item_nbr'].unique()) - set(test['item_nbr'].unique()))
train = train[~train['item_nbr'].isin(items_to_del)]

CPU times: user 12.9 s, sys: 5.74 s, total: 18.7 s
Wall time: 18.7 s


In [11]:
train.shape

(71448006, 5)

In [7]:
train['item_nbr'].nunique()

3840

In [8]:
test['item_nbr'].nunique()

3840

In [14]:
df['item_nbr'].nunique()

3840

#### Объединяем датасеты

In [9]:
%%time
# Объединяем train и test
test['unit_sales'] = 0 
test['unit_sales'] = test['unit_sales'].astype(np.float16)
train.sort_index(axis=1, inplace=True)
test.sort_index(axis=1, inplace=True)
df = pd.concat([train, test])

df['store_nbr'] = df['store_nbr'].astype(np.uint8)
df['item_nbr'] = df['item_nbr'].astype(np.uint32)
df['onpromotion'] = df['onpromotion'].astype(np.uint8)

print(train.shape, test.shape, df.shape)
del train, test
gc.collect()

(71448006, 5) (2486416, 5) (73934422, 5)
CPU times: user 2.01 s, sys: 3.37 s, total: 5.38 s
Wall time: 5.37 s


#### Заполняем нулевые продажи

In [10]:
%%time
df = df.groupby(['store_nbr', 'item_nbr']).apply(lambda x: x.set_index('date').resample('1D').first())
df = df.drop(['store_nbr', 'item_nbr'], axis=1).reset_index().fillna(0)
print(df.shape)

(105930418, 5)
CPU times: user 12min 55s, sys: 1min 5s, total: 14min 1s
Wall time: 14min


#### Сливаем с остальными таблицами

In [11]:
family = {'GROCERY I': 1, 'CLEANING': 2, 'BREAD/BAKERY': 3, 'DELI': 4, 'POULTRY': 5, 'EGGS': 6,
       'PERSONAL CARE': 7, 'LINGERIE': 8, 'BEVERAGES': 9, 'AUTOMOTIVE': 10, 'DAIRY': 11,
       'GROCERY II': 12, 'MEATS': 13, 'FROZEN FOODS': 14, 'HOME APPLIANCES': 15, 'SEAFOOD': 16,
       'PREPARED FOODS': 17, 'LIQUOR,WINE,BEER': 18, 'BEAUTY': 19, 'HARDWARE': 20,
       'LAWN AND GARDEN': 21, 'PRODUCE': 22, 'HOME AND KITCHEN II': 23,
       'HOME AND KITCHEN I': 24, 'MAGAZINES': 25, 'HOME CARE': 26, 'PET SUPPLIES': 27,
       'BABY CARE': 28, 'SCHOOL AND OFFICE SUPPLIES': 29,
       'PLAYERS AND ELECTRONICS': 30, 'CELEBRATION': 31, 'LADIESWEAR': 32, 'BOOKS': 33}
# Товары
types = {'item_nbr': np.uint32,
         'perishable': np.uint8,
         'class' : np.uint16
        }
items = pd.read_csv('data/items.csv.gz', compression='gzip', dtype = types)

items['family'] = items['family'].map(family)
items['family'] = items['family'].astype(np.uint8)

df = df.merge(items, on=['item_nbr'], how='left')

In [12]:
%%time
# Магазины
types = {'cluster': np.uint8,
         'store_nbr': np.uint8,
        }
stores = pd.read_csv('data/stores.csv.gz', compression='gzip' , dtype = types)

stores['city'] = stores['city'].map(city)
stores['state'] = stores['state'].map(state)
stores['type'] = stores['type'].map(store_type)

stores['city'] = stores['city'].astype(np.uint8)
stores['state'] = stores['state'].astype(np.uint8)
stores['type'] = stores['type'].astype(np.uint8)

df = df.merge(stores, on=['store_nbr'], how='left')

CPU times: user 17.3 s, sys: 29.2 s, total: 46.5 s
Wall time: 46.5 s


In [ ]:
# df = df.merge(oil, on=['date'], how='left')
# df = df.merge(transactions, on=['date', 'store_nbr', 'item_nbr'], how='left')

#### Обработка даты и праздников

In [14]:
%%time
# df['year'] = df['date'].apply(lambda x: x.year-2000)
# df['year'] = df['year'].astype(np.uint8)

df['month'] = df['date'].apply(lambda x: x.month)
df['month'] = df['month'].astype(np.uint8)

df['day'] = df['date'].apply(lambda x: x.day)
df['day'] = df['day'].astype(np.uint8)

# df['dayofyear'] = df['date'].apply(lambda x: x.dayofyear)
# df['dayofyear'] = df['dayofyear'].astype(np.uint16)

# df['weekofyear'] = df['date'].apply(lambda x: x.weekofyear)
# df['weekofyear'] = df['weekofyear'].astype(np.uint8)

df['dayofweek'] = df['date'].apply(lambda x: x.dayofweek)
df['dayofweek'] = df['dayofweek'].astype(np.uint8)

# df['days_in_month'] = df['date'].apply(lambda x: x.days_in_month)
# df['days_in_month'] = df['days_in_month'].astype(np.uint8)

# df['quarter'] = df['date'].apply(lambda x: x.quarter)
# df['quarter'] = df['quarter'].astype(np.uint8)

CPU times: user 23min 7s, sys: 1min 33s, total: 24min 40s
Wall time: 24min 27s


In [3]:
for i in range(-3, 4):
    print(i)

-3
-2
-1
0
1
2
3


In [15]:
# Праздники
holiday_events = pd.read_csv('data/holidays_events.csv.gz', compression='gzip', parse_dates = ['date'], 
                             infer_datetime_format = True)
holiday_events['transferred'] = holiday_events['transferred'].astype(np.uint8)
holiday_events.drop(264, inplace=True)

# Создаем выходные и праздники
holiday_events['dayoff'] = 0
mask = (holiday_events['type'] == 'Holiday') & (holiday_events['transferred'] == 0)
holiday_events.loc[mask, 'dayoff'] = 1
mask = (holiday_events['type'] == 'Transfer') | (holiday_events['type'] == 'Additional') | \
       (holiday_events['type'] == 'Bridge')
holiday_events.loc[mask, 'dayoff'] = 1

# Ивенты
holiday_events['event'] = 0
mask = (holiday_events['type'] == 'Event')
holiday_events.loc[mask, 'event'] = 1

# Перенесенные праздники как ивенты
mask = (holiday_events['type'] == 'Holiday') & (holiday_events['transferred'] == 0)
holiday_events.loc[mask, 'event'] = 1

holiday_events['dayoff'] = holiday_events['dayoff'].astype(np.uint8)
holiday_events['event'] = holiday_events['event'].astype(np.uint8)

In [16]:
print(df.shape)
#Сначала по городам
holiday_temp = holiday_events[(holiday_events['locale'] == 'Local')&(holiday_events['dayoff'] == 1)]
holiday_temp['locale_name'] = holiday_temp['locale_name'].map(city)
holiday_temp['locale_name'] = holiday_temp['locale_name'].astype(np.uint8)

holiday_temp.drop(['type', 'locale', 'description', 'transferred', 'event'], axis=1, inplace=True)
holiday_temp = holiday_temp.rename(columns={'locale_name': 'city'})

df = df.merge(holiday_temp, on=['date', 'city'], how='left')
df.rename(columns={'dayoff': 'dayoff1'}, inplace=True)
df['dayoff1'].fillna(0, inplace=True)
df['dayoff1'] = df['dayoff1'].astype(np.uint8)
print(df.shape)

# Потом по штатам
holiday_temp = holiday_events[(holiday_events['locale'] == 'Regional')&(holiday_events['dayoff'] == 1)]
holiday_temp['locale_name'] = holiday_temp['locale_name'].map(state)
holiday_temp['locale_name'] = holiday_temp['locale_name'].astype(np.uint8)

holiday_temp.drop(['type', 'locale', 'description', 'transferred', 'event'], axis=1, inplace=True)
holiday_temp = holiday_temp.rename(columns={'locale_name': 'state'})

df = df.merge(holiday_temp, on=['date', 'state'], how='left')
df.rename(columns={'dayoff': 'dayoff2'}, inplace=True)
df['dayoff2'].fillna(0, inplace=True)
df['dayoff2'] = df['dayoff2'].astype(np.uint8)
print(df.shape)

# Потом национальные
holiday_temp = holiday_events[(holiday_events['locale'] == 'National')&(holiday_events['dayoff'] == 1)]
holiday_temp.drop(['type', 'locale', 'description', 'transferred', 'locale_name', 'event'], axis=1, inplace=True)

df = df.merge(holiday_temp, on=['date'], how='left')
df.rename(columns={'dayoff': 'dayoff3'}, inplace=True)
df['dayoff3'].fillna(0, inplace=True)
df['dayoff3'] = df['dayoff3'].astype(np.uint8)
print(df.shape)
# Потом Events
holiday_temp = holiday_events[holiday_events['event'] == 1]
holiday_temp.drop(['type', 'locale', 'description', 'transferred', 'locale_name', 'dayoff'], axis=1, inplace=True)
holiday_temp.drop_duplicates(inplace=True)

df = df.merge(holiday_temp, on=['date'], how='left')
df['event'].fillna(0, inplace=True)
df['event'] = df['event'].astype(np.uint8)
print(df.shape)

# Сливаем dayoff в одну колонку
df['holiday'] = df['dayoff1']+df['dayoff2']+df['dayoff3']
df.drop(['dayoff1', 'dayoff2', 'dayoff3'],  axis=1, inplace=True)
df['holiday'] = df['holiday'].apply(lambda x: 1 if x>1 else x)
df['holiday'] = df['holiday'].astype(np.uint8)
df['dayoff'] = df['holiday']

del holiday_temp
gc.collect()

(105930418, 15)
(105930418, 16)
(105930418, 17)
(105930418, 18)
(105930418, 19)


42

In [17]:
# Добавляем выходные
mask = (df['dayofweek'] == 5) | (df['dayofweek'] == 6)
df.loc[mask, 'dayoff'] = 1

df['dayoff'] = df['dayoff'].astype(np.uint8)

In [18]:
df.shape

(105930418, 18)

In [48]:
df.to_pickle('df.pkl') 

# Статистики

#### Для продаж

In [3]:
df = pd.read_pickle('df.pkl')
statistics = df.filter(items=['store_nbr', 'item_nbr', 'date', 'unit_sales'])
statistics['unit_sales'] = statistics['unit_sales'].astype(np.float32)
print(statistics.shape)
del df

(105930418, 4)


In [4]:
# 21, 28, 56
statistics = statistics[statistics['date']>='2016-04-01']
# #3, 7, 14
statistics = statistics[statistics['date']>='2016-05-15']

In [4]:
%%time

# Здесь мы храним сгруппированый датасет с установленным индексом по дате. По нему вычисляем статистики
grouped = statistics.groupby(['store_nbr', 'item_nbr']).apply(lambda x: x.set_index('date')) \
                                                       .groupby(level=['store_nbr', 'item_nbr'])['unit_sales']
gc.collect()

CPU times: user 2min 58s, sys: 49 s, total: 3min 47s
Wall time: 3min 49s


#### Для промо

In [2]:
df = pd.read_pickle('df.pkl')
statistics = df.filter(items=['store_nbr', 'item_nbr', 'date', 'onpromotion'])
del df

statistics = statistics[statistics['date']>='2016-06-29']

In [3]:
%%time

# Здесь мы храним сгруппированый датасет с установленным индексом по дате. По нему вычисляем статистики
grouped = statistics.groupby(['store_nbr', 'item_nbr']).apply(lambda x: x.set_index('date')) \
                                                       .groupby(level=['store_nbr', 'item_nbr'])['onpromotion']
gc.collect()

CPU times: user 3min 2s, sys: 30.3 s, total: 3min 32s
Wall time: 3min 32s


#### Только один раз - предыдущий день

In [5]:
%%time
# UNIT_SALES

statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21)).reset_index().rename(columns={'unit_sales': 'prev_day'}), 
                      on=['store_nbr', 'item_nbr', 'date'], how='left')
statistics = statistics[statistics['date']>='2016-06-29']
statistics.fillna(0, inplace=True)
statistics['prev_day'] = statistics['prev_day'].astype(np.float16)
print(statistics.shape)

(61924713, 5)
CPU times: user 3min 58s, sys: 2min 9s, total: 6min 7s
Wall time: 6min 6s


In [ ]:
%%time
# Промо на 16 дней назад

statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift()).reset_index().rename(columns={'onpromotion': 'prev_promo'}), 
                      on=['store_nbr', 'item_nbr', 'date'], how='left')
statistics = statistics[statistics['date']>='2016-06-29']
statistics.fillna(0, inplace=True)
statistics['prev_day'] = statistics['prev_day'].astype(np.float16)
print(statistics.shape)

In [ ]:
# Промо на 16 дней вперед
for i in range(16): 
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(-(i+1))).reset_index().rename(columns={'onpromotion': 'promo_'+str(i+1)+'_next'}), on=['store_nbr', 'item_nbr', 'date'], how='left') 

    statistics['promo_'+str(i+1)+'_next'].fillna(0, inplace=True)
    statistics['promo_'+str(i+1)+'_next'] = statistics['promo_'+str(i+1)+'_next'].astype(np.uint8)
    print(str(i+1)+' days')

    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

1 days
2 days
3 days
4 days
5 days
6 days
7 days
8 days
9 days
10 days
11 days
12 days
13 days
14 days
15 days
16 days
(61924713, 20)
dataset uses  2.19153155945  GB


#### Основной блок

In [5]:
%%time
# MED MEAN

for i in [3, 7, 14]: #3, 7, 14, 21, 28, 56, 84, 168, 336
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21).rolling(window=i, min_periods=1) \
                                                    .agg({'mean_'+str(i)+'_days': 'mean', 'med_'+str(i)+'_days': 'median'})) \
                                             .reset_index(), on=['store_nbr', 'item_nbr', 'date'], how='left') 

    statistics.fillna(0, inplace=True)
    statistics['mean_'+str(i)+'_days'] = statistics['mean_'+str(i)+'_days'].astype(np.float16)
    statistics['med_'+str(i)+'_days'] = statistics['med_'+str(i)+'_days'].astype(np.float16) 

    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

(61924713, 10)
dataset uses  2.13385967631  GB
CPU times: user 31min 35s, sys: 6min 41s, total: 38min 17s
Wall time: 38min 16s


In [5]:
%%time
# QUANT 0.1 / 0.9
WINDOW=336
for i in [.05, .25, .75, .95]: #3, 7, 14, 21, 28, 56, 84, 168, 336
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21).rolling(window=WINDOW, min_periods=1).quantile(i)) \
     .reset_index().rename(columns={'unit_sales': 'Q'+str(i)+'_'+str(WINDOW)}), on=['store_nbr', 'item_nbr', 'date'], how='left') 

    statistics['Q'+str(i)+'_'+str(WINDOW)].fillna(0, inplace=True)
    statistics['Q'+str(i)+'_'+str(WINDOW)] = statistics['Q'+str(i)+'_'+str(WINDOW)].astype(np.float16)
    print(str(i)+' quantile')

    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

0.05 quantile
0.25 quantile
0.75 quantile
0.95 quantile
(61924713, 8)
dataset uses  1.90317214373  GB
CPU times: user 48min 52s, sys: 9min 33s, total: 58min 26s
Wall time: 58min 57s


In [5]:
%%time
# KURT SKEW

for i in [7, 14, 252]: #3, 7, 14, 21, 28, 56, 84, 168, 252, 336
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21).rolling(window=i, min_periods=1) \
                                                    .agg({'kurt_'+str(i)+'_days': 'kurt', 'skew_'+str(i)+'_days': 'skew'})) \
                                             .reset_index(), on=['store_nbr', 'item_nbr', 'date'], how='left') 
    statistics.fillna(0, inplace=True)
    statistics['kurt_'+str(i)+'_days'] = statistics['kurt_'+str(i)+'_days'].astype(np.float16)
    statistics['skew_'+str(i)+'_days'] = statistics['skew_'+str(i)+'_days'].astype(np.float16) 

    print(str(i)+' days statistics')
    
    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

7 days statistics
14 days statistics
252 days statistics
(61924713, 10)
dataset uses  2.13385967631  GB
CPU times: user 29min 32s, sys: 6min 53s, total: 36min 26s
Wall time: 36min 25s


In [ ]:
%%time
# STD HMEAN

def hmean(x):
    return st.hmean(x[~np.isnan(x) & ~np.equal(x, 0)])

for i in [3, 7, 14]: #3, 7, 14, 21, 28, 56, 84, 168, 336
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21).rolling(window=i, min_periods=1) \
                                                    .agg({'std_'+str(i)+'_days': 'std', 'hmean_'+str(i)+'_days': hmean})) \
                                             .reset_index(), on=['store_nbr', 'item_nbr', 'date'], how='left') 
    statistics.fillna(0, inplace=True)
    statistics['std_'+str(i)+'_days'] = statistics['std_'+str(i)+'_days'].astype(np.float16)
    statistics['hmean_'+str(i)+'_days'] = statistics['hmean_'+str(i)+'_days'].astype(np.float16) 

    print(str(i)+' days statistics')
    print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")
    
    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

#### Промо

In [ ]:
%%time
# Только один раз

statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift(21)).reset_index().rename(columns={'unit_sales': 'prev_day'}), 
                      on=['store_nbr', 'item_nbr', 'date'], how='left')
statistics = statistics[statistics['date']>='2016-06-29']
statistics.fillna(0, inplace=True)
statistics['prev_day'] = statistics['prev_day'].astype(np.float16)
print(statistics.shape)

In [5]:
%%time
# PROMO

for i in [7, 14, 21]: #3, 7, 14, 21, 28, 56, 84, 168, 336
    statistics = pd.merge(statistics, grouped.apply(lambda x: x.shift().rolling(window=i, min_periods=1).sum()) \
     .reset_index().rename(columns={'onpromotion': 'promo_'+str(i)+'_prdays'}), on=['store_nbr', 'item_nbr', 'date'], how='left') 

    statistics['promo_'+str(i)+'_prdays'].fillna(0, inplace=True)
    statistics['promo_'+str(i)+'_prdays'] = statistics['promo_'+str(i)+'_prdays'].astype(np.uint8)
    print(str(i)+' days')

    gc.collect()
statistics = statistics[statistics['date']>='2016-06-29']
print(statistics.shape)
print("dataset uses ", statistics.memory_usage(index=True).sum() / 1024**3, " GB")

7 days
14 days
21 days
(61924713, 7)
dataset uses  1.44179707859  GB
CPU times: user 16min 50s, sys: 7min 5s, total: 23min 55s
Wall time: 23min 55s


#### Сохранение Продажи

In [19]:
statistics = pd.read_pickle('statistics_sh.pkl')

In [20]:
for i in [7, 14, 21, 28]:
    unwanted = statistics.columns[statistics.columns.str.endswith(str(i)+'_days')]
    statistics.drop(unwanted, axis=1, inplace=True)

In [10]:
unwanted = statistics.columns[statistics.columns.str.startswith('Q0.05')]
statistics.drop(unwanted, axis=1, inplace=True)

In [14]:
statistics.drop(['onpromotion'], axis=1, inplace=True)

In [15]:
statistics.columns

Index(['store_nbr', 'item_nbr', 'date', 'prev_day', 'med_84_days',
       'mean_84_days', 'med_168_days', 'mean_168_days', 'mean_336_days',
       'med_336_days', 'mean_21_days', 'mean_28_days', 'mean_56_days',
       'med_56_days', 'mean_3_days', 'mean_7_days', 'mean_14_days',
       'skew_168_days', 'kurt_168_days', 'kurt_336_days', 'skew_336_days',
       'skew_252_days', 'kurt_252_days', 'Q0.75_21', 'Q0.25_56', 'Q0.25_84',
       'Q0.75_84', 'Q0.75_168', 'Q0.25_336', 'Q0.75_336', 'Q0.95_336',
       'std_84_days', 'hmean_84_days', 'hmean_168_days', 'std_168_days',
       'hmean_336_days', 'std_336_days', 'std_56_days', 'hmean_56_days',
       'promo_7_prdays', 'promo_14_prdays', 'promo_21_prdays', 'promo_1_next',
       'promo_2_next', 'promo_3_next', 'promo_4_next', 'promo_5_next',
       'promo_6_next', 'promo_7_next', 'promo_8_next', 'promo_9_next',
       'promo_10_next', 'promo_11_next', 'promo_12_next', 'promo_13_next',
       'promo_14_next', 'promo_15_next', 'promo_16_next'

In [21]:
# stats = pd.read_pickle('statistics.pkl')
print(stats.shape)
stats = pd.merge(stats, statistics, on=['store_nbr', 'item_nbr', 'date'], how='left')
print(stats.shape)



(61924713, 59)
(61924713, 66)


In [16]:
statistics.to_pickle('statistics.pkl')

In [23]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61924713 entries, 0 to 61924712
Data columns (total 66 columns):
store_nbr          uint8
item_nbr           uint32
date               datetime64[ns]
prev_day           float16
med_84_days        float16
mean_84_days       float16
med_168_days       float16
mean_168_days      float16
mean_336_days      float16
med_336_days       float16
mean_21_days       float16
mean_28_days       float16
mean_56_days       float16
med_56_days        float16
mean_3_days        float16
mean_7_days        float16
mean_14_days       float16
skew_168_days      float16
kurt_168_days      float16
kurt_336_days      float16
skew_336_days      float16
skew_252_days      float16
kurt_252_days      float16
Q0.75_21           float16
Q0.25_56           float16
Q0.25_84           float16
Q0.75_84           float16
Q0.75_168          float16
Q0.25_336          float16
Q0.75_336          float16
Q0.95_336          float16
std_84_days        float16
hmean_84_days    

In [ ]:
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(unwanted, axis=1)

In [11]:
statistics = pd.read_pickle('statistics.pkl')

In [13]:
statistics.columns

Index(['store_nbr', 'item_nbr', 'date', 'prev_day', 'med_84_days',
       'mean_84_days', 'med_168_days', 'mean_168_days', 'mean_336_days',
       'med_336_days', 'mean_21_days', 'mean_28_days', 'mean_56_days',
       'med_56_days', 'mean_3_days', 'mean_7_days', 'mean_14_days',
       'skew_168_days', 'kurt_168_days', 'kurt_336_days', 'skew_336_days',
       'skew_252_days', 'kurt_252_days', 'Q0.75_21', 'Q0.25_56', 'Q0.25_84',
       'Q0.75_84', 'Q0.75_168', 'Q0.25_336', 'Q0.75_336', 'Q0.95_336',
       'std_84_days', 'hmean_84_days', 'hmean_168_days', 'std_168_days',
       'hmean_336_days', 'std_336_days', 'std_56_days', 'hmean_56_days',
       'promo_7_prdays', 'promo_14_prdays', 'promo_21_prdays', 'onpromotion',
       'promo_1_next', 'promo_2_next', 'promo_3_next', 'promo_4_next',
       'promo_5_next', 'promo_6_next', 'promo_7_next', 'promo_8_next',
       'promo_9_next', 'promo_10_next', 'promo_11_next', 'promo_12_next',
       'promo_13_next', 'promo_14_next', 'promo_15_next', 

In [20]:
# Для создания первого файла статистики
statistics.drop(['onpromotion'], axis=1).to_pickle('statistics_pr.pkl') 

In [9]:
statistics.drop(['onpromotion'], axis=1, inplace=True)
print(statistics.shape)
stats = pd.read_pickle('statistics.pkl')
print(stats.shape)
stats = pd.merge(stats, statistics, on=['store_nbr', 'item_nbr', 'date'], how='left')
print(stats.shape)

stats.to_pickle('statistics.pkl') 

(61924713, 6)
(61924713, 43)
(61924713, 46)


In [ ]:
statistics.drop(['store_nbr', 'item_nbr', 'date', 'unit_sales'], axis=1, inplace=True)
stats = pd.read_pickle('statistics_ks.pkl')
stats = pd.concat([stats, statistics], axis=1, ignore_index=True)
stats.to_pickle('statistics_ks.pkl') 

In [7]:
statistics.drop(['store_nbr', 'item_nbr', 'date', 'unit_sales'], axis=1, inplace=True)
stats = pd.read_pickle('statistics_sh.pkl')
stats = pd.concat([stats, statistics], axis=1, ignore_index=True)
stats.to_pickle('statistics_sh.pkl') 

#### Сохранение Промо

In [ ]:
# Для создания первого файла статистики
statistics.drop(['onpromotion'], axis=1).to_pickle('statistics.pkl') 

In [ ]:
statistics = statistics[statistics['date']>='2016-06-29']
statistics.drop(['store_nbr', 'item_nbr', 'date', 'onpromotion'], axis=1, inplace=True)
stats = pd.read_pickle('statistics.pkl')
stats = pd.concat([stats, statistics], axis=1)
stats.to_pickle('statistics_pr.pkl') 

# Таргет кодирование

#### Кодирование средним (ВАЛИДАЦИЯ)

In [2]:
# Если валидация
SUBMIT = False
TRAIN_START = '2017-03-01'
TRAIN_END = '2017-07-11'
TEST_START = '2017-07-12'
TEST_END = '2017-07-27'

In [2]:
SUBMIT = True
TRAIN_START = '2017-05-01'
TRAIN_END = '2017-08-15'
TEST_START = '2017-08-16'

In [3]:
%%time
# Разбиваем на трейн и тест
df = pd.read_pickle('df.pkl')
train = df[(df['date']>=TRAIN_START)&(df['date']<=TRAIN_END)]
if SUBMIT:
    test = df[df['date']>=TEST_START]
else:
    test = df[(df['date']>=TEST_START)&(df['date']<=TEST_END)]

del df
train.sort_values(by=['date', 'store_nbr', 'item_nbr'], inplace=True)
test.sort_values(by=['date', 'store_nbr', 'item_nbr'], inplace=True)


# Сохраняем веса
weights_train = train['perishable'].values * 0.25 + 1
weights_train = weights_train.tolist()
weights_test = test['perishable'].values * 0.25 + 1
weights_test = weights_test.tolist()

CPU times: user 22.9 s, sys: 23.8 s, total: 46.7 s
Wall time: 51.6 s


In [4]:
# Список всех категориальных признаков
features = ['store_nbr', 'item_nbr', 'date', 'onpromotion', 'unit_sales',
       'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster',
       'month', 'day', 'dayofweek', 'event', 'holiday', 'dayoff']

# Категориальные данные, на которых обучаем
fit_features = ['store_nbr', 'item_nbr', 'onpromotion', 'dayofweek', 'dayoff', 'unit_sales', 'date',
               'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster', 'day', 'event', 'holiday']
cat_features = ['store_nbr', 'item_nbr', 'dayofweek', 'family', 'class', 'city', 'state', 'type', 'cluster',
                'day']

# Которые удаляем
drop_features = list(set(features) - set(fit_features))

# Удаляем ненужные
train.drop(drop_features, axis=1, inplace=True)
test.drop(drop_features, axis=1, inplace=True)

print(train.shape, test.shape)

(16443365, 17) (2486416, 17)


In [5]:
%%time
# Добавляем в датасет статистику и удаляем дату

stats = pd.read_pickle('statistics.pkl')
stats = stats[stats['date']>=TRAIN_START]
train = pd.merge(train, stats, on=['store_nbr', 'item_nbr', 'date'], how='left')
test = pd.merge(test, stats, on=['store_nbr', 'item_nbr', 'date'], how='left')
del stats
train.drop('date', axis=1, inplace=True)
test.drop('date', axis=1, inplace=True)
gc.collect()
print(train.shape, test.shape)
print("dataset uses ", train.memory_usage(index=True).sum() / 1024**3, " GB")
print("dataset uses ", test.memory_usage(index=True).sum() / 1024**3, " GB")

(16443365, 78) (2486416, 78)
dataset uses  1.94488778245  GB
dataset uses  0.294088229537  GB
CPU times: user 45.2 s, sys: 1min 12s, total: 1min 57s
Wall time: 2min 7s


In [6]:
%%time
from sklearn.model_selection import KFold

# На время groupby меняем тип данных (не помещается)
train['unit_sales'] = train['unit_sales'].astype(np.float32)

# X_train_new - для кодированных признаков
X_train_new = train.copy()

for col in cat_features:
    X_train_new[col+'_enc'] = 0
    X_train_new[col+'_enc'] = X_train_new[col+'_enc'].astype(np.float32)
    
cv = KFold(n_splits=10, shuffle = False)

for train_idx, test_idx in cv.split(train):
    X_train, X_test = train.iloc[train_idx], train.iloc[test_idx]

    for col in cat_features:
        X_test[col+'_enc'] = X_test[col].map(X_train.groupby(col)['unit_sales'].mean())
    X_train_new.iloc[test_idx] = X_test

for col in cat_features:
    X_train_new[col+'_enc'] = X_train_new[col+'_enc'].astype(np.float16)
    
prior = X_train_new['unit_sales'].mean()
X_train_new.fillna(prior, inplace=True)
train = X_train_new.copy()
del X_train_new
train['unit_sales'] = train['unit_sales'].astype(np.float16)
gc.collect()

CPU times: user 1min 56s, sys: 2min 14s, total: 4min 10s
Wall time: 4min 10s


In [7]:
for col in cat_features:
    test[col+'_enc'] = test[col].map(train.groupby(col)[col+'_enc'].mean().to_dict())
    
train.drop(cat_features, axis=1, inplace=True)
test.drop(cat_features, axis=1, inplace=True)

In [21]:
WEIGHTS = weights_test

def NWRMSLE(y, pred):
    y = y.clip(0, y.max())
    pred = pred.clip(0, pred.max())
    score = np.nansum(WEIGHTS * ((pred - y) ** 2)) / np.sum(WEIGHTS)
    return np.sqrt(score)

In [ ]:
%%time
np.random.seed(42)

y_train = train['unit_sales'].values
y_test = test['unit_sales'].values
X_train = train.drop('unit_sales', axis=1)
X_test = test.drop('unit_sales', axis=1)
del train, test
gc.collect()

lgb_train = lgb.Dataset(X_train, label=y_train, weight=weights_train)
lgb_eval = lgb.Dataset(X_test, label=y_test, reference=lgb_train, weight=weights_test)

params = {'num_leaves': 2**6 - 1,
    'objective': 'regression',
#     'max_depth': 10,
    'min_data_in_leaf': 300,
    'learning_rate': 0.2,
    'feature_fraction': 0.7,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 2000

print('Start training...')
bst = lgb.train(params,
                lgb_train,
                num_boost_round=MAX_ROUNDS,
                valid_sets=lgb_eval,
                early_stopping_rounds=50,
                verbose_eval=50)

# print('Save model...')
# gbm.save_model('model.txt')
print("\n".join(("%s: %.2f" % x) for x in sorted(zip(X_train.columns.values.tolist(), bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True)))

print('Start predicting...')
# Предикт на валидации
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS)

print('The NWRMSLE is:', NWRMSLE(y_test, y_pred))

Start training...
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's l2: 0.411599
[100]	valid_0's l2: 0.410737
[150]	valid_0's l2: 0.410002
[200]	valid_0's l2: 0.409051
[250]	valid_0's l2: 0.408313
[300]	valid_0's l2: 0.407933
[350]	valid_0's l2: 0.407851
[400]	valid_0's l2: 0.407778
Early stopping, best iteration is:
[395]	valid_0's l2: 0.407663
mean_28_days: 11033902.18
mean_56_days: 8293843.31
mean_21_days: 6243272.98
med_56_days: 2953853.81
mean_14_days: 2426734.01
onpromotion: 1938835.51
hmean_84_days: 1809357.05
hmean_336_days: 1594923.30
mean_84_days: 865529.59
prev_day: 838108.41
hmean_168_days: 661895.71
item_nbr_enc: 524993.34
mean_3_days: 501997.53
dayofweek_enc: 456838.79
med_84_days: 452287.51
Q0.25_56: 442169.10
dayoff: 420409.74
skew_336_days: 409912.37
mean_336_days: 347154.02
mean_168_days: 294947.25
family_enc: 284976.96
Q0.75_168: 263362.43
promo_7_prdays: 235334.66
med_336_days: 195590.45
store_nbr_enc: 194245.10
class_enc: 175861.46
day_en

NameError: name 'NWRMSLE' is not defined

In [22]:
print('The NWRMSLE is:', NWRMSLE(y_test, y_pred))

The NWRMSLE is: 0.638383660572


In [16]:
X_test.drop(['promo_11_next','promo_10_next','promo_9_next','promo_12_next','promo_6_next','promo_8_next',
              'promo_15_next','promo_16_next','promo_5_next','promo_4_next','promo_13_next'], axis=1, inplace=True)

In [ ]:
0.415385

#### Кодирование средним (ОБУЧЕНИЕ)

In [9]:
%%time

y_train = train['unit_sales'].values
# y_test = test['unit_sales'].values
X_train = train.drop('unit_sales', axis=1)
X_test = test.drop('unit_sales', axis=1)
del train, test
gc.collect()

lgb_train = lgb.Dataset(X_train, label=y_train, weight=weights_train)
lgb_test = lgb.Dataset(X_test, label=None, weight=weights_test)

params = {'num_leaves': 2**6 - 1,
    'objective': 'regression',
#     'max_depth': 10,
    'min_data_in_leaf': 300,
    'learning_rate': 0.2,
    'feature_fraction': 0.7,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 600

print('Start training...')
bst = lgb.train(params,
                lgb_train,
                num_boost_round=MAX_ROUNDS)


print('Start predicting...')
y_pred = bst.predict(X_test, num_iteration=MAX_ROUNDS)

Start training...
Start predicting...
CPU times: user 1h 22min 1s, sys: 1min 18s, total: 1h 23min 20s
Wall time: 22min 41s


In [14]:
print("\n".join(("%s: %.2f" % x) for x in sorted(zip(X_train.columns.values.tolist(), bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True)))

mean_28_days: 10948290.45
mean_56_days: 6124674.85
mean_21_days: 5621474.90
mean_14_days: 1709247.01
onpromotion: 1633386.37
hmean_84_days: 1535725.13
hmean_336_days: 1290536.88
med_168_days: 926793.64
mean_84_days: 705365.55
prev_day: 677077.09
mean_168_days: 619366.03
item_nbr_enc: 450468.21
mean_3_days: 402355.83
Q0.25_56: 397736.18
dayofweek_enc: 371267.32
dayoff: 294233.46
med_84_days: 286135.89
hmean_168_days: 262645.01
Q0.75_336: 258543.33
family_enc: 254682.71
Q0.75_168: 231934.33
skew_336_days: 193562.86
med_336_days: 172298.93
promo_7_prdays: 167125.82
class_enc: 160905.57
mean_336_days: 151594.12
day_enc: 137866.15
store_nbr_enc: 133417.62
type_enc: 113855.66
std_336_days: 90290.44
Q0.95_336: 85153.88
cluster_enc: 84407.03
Q0.25_84: 66345.70
promo_168_days: 66115.56
std_168_days: 66098.28
promo_1_next: 63003.63
promo_21_prdays: 57048.29
city_enc: 51992.66
std_84_days: 50963.30
state_enc: 46838.71
hmean_56_days: 46679.50
promo_21_days: 44728.61
promo_56_days: 44162.57
promo_1

#### Expanding mean

In [2]:
import pandas as pd
df = pd.read_pickle('df.pkl')

In [ ]:
holiday_events = pd.read_csv('data/holidays_events.csv.gz', compression='gzip', parse_dates = ['date'], 
                             infer_datetime_format = True)
holiday_events['transferred'] = holiday_events['transferred'].astype(np.uint8)
holiday_events.drop(264, inplace=True)

# Создаем выходные и праздники
holiday_events['dayoff'] = 0
mask = (holiday_events['type'] == 'Holiday') & (holiday_events['transferred'] == 0)
holiday_events.loc[mask, 'dayoff'] = 1
mask = (holiday_events['type'] == 'Transfer') | (holiday_events['type'] == 'Additional') | \
       (holiday_events['type'] == 'Bridge')
holiday_events.loc[mask, 'dayoff'] = 1

In [5]:
mask = (df['holiday'] == 1) & (df['date'] >= '2017-08-16')
df.loc[mask]

,store_nbr,item_nbr,date,onpromotion,unit_sales,family,class,perishable,city,state,type,cluster,month,day,dayofweek,event,holiday,dayoff
42193181,23,99197,2017-08-24,0,0.0,1,1067,0,9,38,4,9,8,24,3,1,1,1
42193974,23,103520,2017-08-24,0,0.0,1,1028,0,9,38,4,9,8,24,3,1,1,1
42194767,23,103665,2017-08-24,0,0.0,3,2712,1,9,38,4,9,8,24,3,1,1,1
42195560,23,105574,2017-08-24,0,0.0,1,1045,0,9,38,4,9,8,24,3,1,1,1
42196353,23,105575,2017-08-24,0,0.0,1,1045,0,9,38,4,9,8,24,3,1,1,1
42197140,23,105577,2017-08-24,0,0.0,1,1045,0,9,38,4,9,8,24,3,1,1,1
42197930,23,105693,2017-08-24,0,0.0,1,1034,0,9,38,4,9,8,24,3,1,1,1
42198723,23,105737,2017-08-24,0,0.0,1,1044,0,9,38,4,9,8,24,3,1,1,1
42199516,23,105857,2017-08-24,0,0.0,1,1092,0,9,38,4,9,8,24,3,1,1,1
42200024,23,108079,2017-08-24,0,0.0,1,1030,0,9,38,4,9,8,24,3,1,1,1


In [6]:
%%time

# На время groupby меняем тип данных (не помещается)
train['unit_sales'] = train['unit_sales'].astype(np.float32)

for col in cat_features:

    cumsum = train.groupby(col)['unit_sales'].cumsum() - train['unit_sales']
    cumcount = train.groupby(col)['unit_sales'].cumcount() + 1
    train[col+'_enc'] = cumsum / cumcount     

    train[col+'_enc'] = train[col+'_enc'].astype(np.float16)
    
    test[col+'_enc'] = test[col].map(train.groupby(col)[col+'_enc'].mean().to_dict())
    test[col+'_enc'] = test[col+'_enc'].astype(np.float16)
    
train.drop(cat_features, axis=1, inplace=True)
test.drop(cat_features, axis=1, inplace=True)

# Проверить сортировку признаков трейн-тест

# Возвращаем тип данных
train['unit_sales'] = train['unit_sales'].astype(np.float16)

print("train dataset uses ", train.memory_usage(index=True).sum() / 1024**3, " GB")
gc.collect()

train dataset uses  2.03677224461  GB
CPU times: user 41 s, sys: 1min 29s, total: 2min 10s
Wall time: 2min 7s


In [17]:
test['unit_sales'] = test['unit_sales'].astype(np.float32)
prior = test['unit_sales'].mean()
test.fillna(prior, inplace=True)
test['unit_sales'] = test['unit_sales'].astype(np.float16)

In [8]:
for col in cat_features:
    print(test[col+'_enc'].isnull().sum())

0
0
0
0
0
0
0
0
0
0


# Обучение

In [ ]:
%%time

y_train = train['unit_sales'].values
X_train = train.drop('unit_sales', axis=1).values
X_test = test.values
del train, test
gc.collect()

lgb_train = lgb.Dataset(X_train, y_train, weight=weights)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,)
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5)

# print('Save model...')
# gbm.save_model('model.txt')

print('Start predicting...')
y_pred = gbm.predict(X_test, num_iteration=1000)
# eval
# print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# Сабмит

In [10]:
test_predict = pd.read_pickle('test_predict.pkl')
test_predict.shape

(2486416, 1)

In [10]:
len(y_pred)

2486416

In [11]:
np.max(np.expm1(y_pred))

1253.9175399715721

In [12]:
test_predict = pd.read_pickle('test_predict.pkl')
test_full = pd.read_csv('data/test.csv.gz', compression='gzip', usecols=[0])
test_predict['unit_sales'] = np.expm1(y_pred)
test_full = test_full.merge(test_predict[['unit_sales']], left_index=True, right_index=True, how='left')
test_full['unit_sales'].fillna(0, inplace=True)

submission = pd.read_csv('data/sample_submission.csv.gz', compression='gzip')
submission['unit_sales'] = test_full['unit_sales'].values
submission['unit_sales'] = submission['unit_sales'].apply(lambda x: 0 if x<0 else x)
submission.to_csv('submission.csv.gz', index=False, compression='gzip')

In [16]:
submission.head(10)

,id,unit_sales
0,125497040,0.190863
1,125497041,0.609509
2,125497042,0.000000
3,125497043,1.209329
4,125497044,1.875669
5,125497045,5.333329
6,125497046,8.897631
7,125497047,0.000000
8,125497048,0.541945
9,125497049,0.103297


In [13]:
submission.head(10)

,id,unit_sales
0,125497040,0.187859
1,125497041,0.548710
2,125497042,0.000000
3,125497043,1.152549
4,125497044,2.109402
5,125497045,5.379028
6,125497046,9.358836
7,125497047,0.000000
8,125497048,0.728287
9,125497049,0.217917
